# VAR calculation for 5 Stock Portfolio

In [12]:
pip install --upgrade yfinance

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# importing relevant library for VAR calculation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm

In [2]:
# importing datetime library
import datetime as dt

In [3]:
# setting years, start date, end date for fetching data from the yfinance library
years = 5
enddate = dt.datetime.now()
startdate = enddate - dt.timedelta(days = 365*years)

In [4]:
# stocks ticker symbol including Apple, Microsoft, Tesla, Google, Nvidia
tickers = ['AAPL','MSFT','TSLA','GOOGL','NVDA']

In [5]:
# using for loop to call the data one by one and storing it in adj_close variable
# we are only going to take adjusted closing price
data = pd.DataFrame()
for i in tickers:
    data[i] = yf.download(i,startdate,enddate)['Close']

print(data)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                  AAPL        MSFT        TSLA       GOOGL        NVDA
Date                                                                  
2020-04-20   67.205101  167.563736   49.757332   62.757462    7.149908
2020-04-21   65.127792  160.633759   45.781334   60.319618    6.713017
2020-04-22   67.003685  166.089661   48.807335   62.621113    7.127491
2020-04-23   66.744026  164.079559   47.042000   63.256073    7.074186
2020-04-24   68.670891  167.075546   48.343334   63.526283    7.213175
...                ...         ...         ...         ...         ...
2025-04-11  198.149994  388.450012  252.309998  157.139999  110.930000
2025-04-14  202.520004  387.809998  252.350006  159.070007  110.709999
2025-04-15  202.139999  385.730011  254.110001  156.309998  112.199997
2025-04-16  194.270004  371.609985  241.550003  153.330002  104.489998
2025-04-17  196.979996  367.779999  241.369995  151.160004  101.489998

[1257 rows x 5 columns]


In [6]:
# getting the daily return of the 5 stocks
log_return = np.log(data/data.shift(1))
log_return = log_return.dropna()
print(log_return)

                AAPL      MSFT      TSLA     GOOGL      NVDA
Date                                                        
2020-04-21 -0.031398 -0.042237 -0.083281 -0.039620 -0.063051
2020-04-22  0.028396  0.033401  0.064004  0.037445  0.059911
2020-04-23 -0.003883 -0.012176 -0.036840  0.010089 -0.007507
2020-04-24  0.028461  0.018095  0.027288  0.004263  0.019457
2020-04-27  0.000706 -0.002869  0.096669 -0.004506  0.025535
...              ...       ...       ...       ...       ...
2025-04-11  0.039792  0.018447 -0.000357  0.027876  0.030758
2025-04-14  0.021814 -0.001649  0.000159  0.012207 -0.001985
2025-04-15 -0.001878 -0.005378  0.006950 -0.017503  0.013369
2025-04-16 -0.039712 -0.037293 -0.050691 -0.019249 -0.071192
2025-04-17  0.013853 -0.010360 -0.000745 -0.014254 -0.029131

[1256 rows x 5 columns]


In [7]:
# lets assume that we have 1 million dollar portfolio and we are equally investing in the 5 stocks
port_val = 1000000
weight = np.array([1/len(tickers)]*len(tickers))
print(weight)

[0.2 0.2 0.2 0.2 0.2]


In [8]:
# multiplying the desginated weight with each stocks and summing it to get the return on the portfolio
hist_return = (log_return * weight).sum(axis =1)
hist_return

Date
2020-04-21   -0.051917
2020-04-22    0.044631
2020-04-23   -0.010063
2020-04-24    0.019512
2020-04-27    0.023107
                ...   
2025-04-11    0.023303
2025-04-14    0.006109
2025-04-15   -0.000888
2025-04-16   -0.043627
2025-04-17   -0.008127
Length: 1256, dtype: float64

In [9]:
# cf is confidence interval 
# using the percentile method to get the value standing at 5th position out of 100 data
# calculating the daily VAR of the portfolio i.e., 5% chance that the portfolio can loose atleast the given amount 
cf = 0.95
var = np.percentile(hist_return,100 - (cf*100))
var_dollar = var * port_val
var_dollar

-34818.20910074116